In [1]:
from ast import literal_eval
import functools
import json
import os
import random
import shutil
import pdb

# Scienfitic packages
import numpy as np
import pandas as pd
import torch
import datasets
from torch import cuda
torch.set_grad_enabled(False)
from tqdm  import tqdm

# Visuals
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(context="notebook",
        rc={"font.size":16,
            "axes.titlesize":16,
            "axes.labelsize":16,
            "xtick.labelsize": 16.0,
            "ytick.labelsize": 16.0,
            "legend.fontsize": 16.0})
palette_ = sns.color_palette("Set1")
palette = palette_[2:5] + palette_[7:]
sns.set_theme(style='whitegrid')

# Utilities

from general_utils import (
  ModelAndTokenizer,
  make_inputs,
  decode_tokens,
  find_token_range,
  predict_from_input,
)

from patchscopes_utils import *

from tqdm import tqdm
tqdm.pandas()

In [2]:
model_to_hook = {
    "EleutherAI/pythia-6.9b": set_hs_patch_hooks_neox,
    "/data3/MODELS/EleutherAI_pythia-12b": set_hs_patch_hooks_neox_batch,
    "meta-llama/Llama-2-13b-hf": set_hs_patch_hooks_llama,
    "lmsys/vicuna-7b-v1.5": set_hs_patch_hooks_llama,
    "./stable-vicuna-13b": set_hs_patch_hooks_llama,
    "CarperAI/stable-vicuna-13b-delta": set_hs_patch_hooks_llama,
    "/data3/MODELS/gpt-j-6b": set_hs_patch_hooks_gptj_batch,
    "/data3/MODELS/Meta-Llama-3-8B-Instruct/":set_hs_patch_hooks_llama,
    "/data3/MODELS/llama2-hf/llama-2-13b-chat":set_hs_patch_hooks_llama_batch,
    "/data3/MODELS/llama2-hf/llama-2-13b":set_hs_patch_hooks_llama_batch,
    "/data3/MODELS/Mistral-7B-Instruct-v0.2":set_hs_patch_hooks_mistral_batch,
    "/data3/MODELS/Qwen/Qwen2.5-7B-Instruct" : set_hs_patch_hooks_qwen_batch,
}


In [3]:
# Load model

# 0-shot with GPT-J
model_name = "/data3/MODELS/llama2-hf/llama-2-13b"
sos_tok = False

if "13b" in model_name or "12b" in model_name:
    torch_dtype = torch.float16
else:
    torch_dtype = None

my_device = torch.device("cuda:5")

mt = ModelAndTokenizer(
    model_name,
    low_cpu_mem_usage=False,
    torch_dtype=torch_dtype,
    device=my_device,
)
mt.set_hs_patch_hooks = model_to_hook[model_name]
mt.model.eval()

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
      

In [4]:
def run_experiment(task_type, task_name, data_dir, output_dir, batch_size=1, n_samples=-1,
                   save_output=True, replace=False, only_correct=True, is_icl=True):
    fdir_out = f"{output_dir}/{task_type}"
    fname_out = f"{fdir_out}/{task_name}_only_correct_{only_correct}.pkl"
    # if not replace and os.path.exists(fname_out):
    #     print(f"File {fname_out} exists. Skipping...")
    #     return
    print(f"Running experiment on {task_type}/{task_name}...")
    df = pd.read_pickle(f"{data_dir}/{task_type}/{task_name}.pkl")
    pd.set_option('display.max_columns',1000)   # 设置最大显示列数的多少
    pd.set_option('display.width',1000)         # 设置宽度,就是说不换行,比较好看数据
    if only_correct:
        df = df[df["is_correct_baseline"]].reset_index(drop=True)
    # Dropping empty prompt sources. This is an artifact of saving and reloading inputs
    df = df[~df["prompt_source"].apply(lambda x: isinstance(x, float))].reset_index(drop=True)
    # Dropping prompt sources with \n. pandas read_pickle is not able to handle them properly and drops the rest of the input.
    df = df[~df["prompt_source"].str.contains('\n')].reset_index(drop=True)
    # After manual inspection, this example seems to have tokenization issues. 0Dropping.
    if task_name == "star_constellation":
        df = df[~df["prompt_source"].str.contains("service")].reset_index(drop=True)
    elif task_name == "object_superclass":
        df = df[~df["prompt_source"].str.contains("Swainson ’ s hawk and the prairie")].reset_index(drop=True)
        
    # 对于相同的target_baseline，只保留第一个
    df = df.drop_duplicates(subset=["target_baseline"]).reset_index(drop=True)
    print(f"\tNumber of samples: {len(df)}")

    def evaluate(mt,
                 df,
                 batch_size=1,  # 修改为1
                 max_gen_len=40,
                 transform=None,
                 only_correct=True,
                 is_icl=False
                 ):
        def evaluate_single_batch(batch_df):
            batch_size = len(batch_df)
            target_baseline_batch = np.array(batch_df["target_baseline"])
            object_batch = np.array(batch_df["object"])
            inp_target = make_inputs(mt.tokenizer, target_baseline_batch, mt.device)
            seq_len = len(inp_target["input_ids"][0])

            
            output_toks = mt.model.generate(
            inp_target["input_ids"],
            max_length=seq_len + max_gen_len,
            pad_token_id=mt.model.generation_config.eos_token_id,
            # pad_token_id=mt.tokenizer.eos_token_id,
            )[:, seq_len:]
            
            generations_patched = decode_tokens(mt.tokenizer, output_toks)
            if is_icl:
                prefix = batch_df["prefix"].iloc[0]

                def _crop_by_prefix(generations, prefix):
                    concatenated_str = " ".join(generations)
                    _pos = concatenated_str.find(prefix)
                    return concatenated_str[:_pos]

                generations_patched_postprocessed = np.array([
                    _crop_by_prefix(generations_patched[i], prefix)
                    for i in range(batch_size)
                ])
            else:
                generations_patched_postprocessed = np.array(
                [" ".join(generations_patched[i]) for i in range(batch_size)]
            )
            
            is_correct_patched = np.array([
                object_batch[i].replace(" ", "").lower()
                in generations_patched_postprocessed[i].replace(" ", "").lower() 
                for i in range(batch_size)
            ])
            
            result = {
                "is_correct_patched": is_correct_patched,
                "generations_patched": generations_patched_postprocessed,
            }
            return result
        
        results = {}
        n_batches = len(df) // batch_size
        if len(df) % batch_size != 0:
            n_batches += 1
        for i in tqdm(range(n_batches)):
            batch_df = df.iloc[i * batch_size:(i + 1) * batch_size] ## iloc函数通过行号来取行数据
            result = evaluate_single_batch(batch_df)
            for k, v in result.items():
                if k not in results:
                    results[k] = []
                results[k].extend(v)
        return results
    
    eval_results = evaluate(mt, df, batch_size=batch_size)
    results_df = df.head(len(eval_results["is_correct_patched"]))
    accuracy = np.mean(eval_results["is_correct_patched"])
    print (f"{task_name} Accuracy: {accuracy}")
    for key, value in eval_results.items():
        results_df[key] = list(value)

    if save_output:
        fdir_out = f"{output_dir}/{task_type}"
        if not os.path.exists(fdir_out):
            os.makedirs(fdir_out)
        results_df.to_csv(f"{fdir_out}/{task_name}_only_correct_{only_correct}.tsv", sep="\t")
        results_df.to_pickle(f"{fdir_out}/{task_name}_only_correct_{only_correct}.pkl")


In [5]:
for task_type in ["factual"]:
    for fname in tqdm(os.listdir(f"./preprocessed_data/llama2/{task_type}")):
        if fname.endswith('.pkl'):
            task_name = fname[:-4]
        else:
            continue
        print(f"Processing {fname}...")
        run_experiment(task_type, task_name, "./preprocessed_data/llama2", "./single_problem_output/llama2_13b", only_correct=False, is_icl=False)
        ## accuracy表示is_correct_patched的准确率
        
        

  0%|          | 0/22 [00:00<?, ?it/s]

Processing superhero_person.pkl...
Running experiment on factual/superhero_person...
	Number of samples: 79


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 79/79 [01:43<00:00,  1.31s/it]
/tmp/ipykernel_79992/342515986.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
/tmp/ipykernel_79992/342515986.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
  5%|▍ 

superhero_person Accuracy: 0.08860759493670886
Processing company_ceo.pkl...
Running experiment on factual/company_ceo...
	Number of samples: 202


100%|██████████| 202/202 [04:24<00:00,  1.31s/it]
/tmp/ipykernel_79992/342515986.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
/tmp/ipykernel_79992/342515986.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
  9%|▉         | 2/22 [06:07<1:05:58, 197.92s/it]

...

country_capital_city Accuracy: 0.875
Processing country_currency.pkl...
Running experiment on factual/country_currency...
	Number of samples: 30


100%|██████████| 30/30 [00:39<00:00,  1.31s/it]
/tmp/ipykernel_79992/342515986.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
/tmp/ipykernel_79992/342515986.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
 86%|████████▋ | 19/22 [29:07<01:16, 25.42s/it]

country_currency Accuracy: 0.9666666666666667
Processing superhero_archnemesis.pkl...
Running experiment on factual/superhero_archnemesis...
	Number of samples: 87


100%|██████████| 87/87 [01:54<00:00,  1.32s/it]
/tmp/ipykernel_79992/342515986.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
/tmp/ipykernel_79992/342515986.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
100%|██████████| 22/22 [31:02<00:00, 84.64s/it]

superhero_archnemesis Accuracy: 0.3218390804597701


In [2]:

l = 0
for task_type in ["factual"]:
    for fname in tqdm(os.listdir(f"./preprocessed_data/gpt-j/{task_type}")):
        if fname.endswith('.pkl'):
            task_name = fname[:-4]
        else:
            continue
        df = pd.read_pickle(f"./preprocessed_data/gpt-j/factual/{task_name}.pkl")
        df = df.drop_duplicates(subset=["target_baseline"]).reset_index(drop=True)
        l += len(df)
        


print(l)
        

100%|██████████| 11/11 [00:00<00:00, 792.86it/s]

1396


In [ ]:

input_path = "./original_result/llama2/evaluation"
input_path = "./multi_patch_output/mistral/5_patch/evaluation"
for fname in tqdm(os.listdir(input_path)):
    if fname.endswith('.pkl') and "False" in fname:
        task_name = fname[:-4]
    else:
        continue
    # print(f"Processing {fname}...")
    df = pd.read_pickle(f"{input_path}/{task_name}.pkl")
    # 对于相同的subject，只要有一个is_correct_patched为True，就认为是正确的
    df = df.groupby("subject").agg({"is_correct_patched": "max"}).reset_index()

    accuracy = np.mean(df["is_correct_patched"])
    print(f"{task_name} Accuracy: {accuracy}")

  0%|          | 0/22 [00:00<?, ?it/s]

superhero_person_only_correct_False Accuracy: 0.4430379746835443
country_capital_city_only_correct_False Accuracy: 0.9166666666666666
product_by_company_only_correct_False Accuracy: 0.848780487804878
food_from_country_only_correct_False Accuracy: 0.9333333333333333
country_largest_city_only_correct_False Accuracy: 1.0
country_currency_only_correct_False Accuracy: 1.0


 64%|██████▎   | 14/22 [00:00<00:00, 89.65it/s]

person_plays_position_in_sport_only_correct_False Accuracy: 0.7994722955145118
person_plays_pro_sport_only_correct_False Accuracy: 0.9694656488549618
star_constellation_only_correct_False Accuracy: 0.43859649122807015


100%|██████████| 22/22 [00:00<00:00, 79.62it/s]

superhero_archnemesis_only_correct_False Accuracy: 0.4827586206896552
company_ceo_only_correct_False Accuracy: 0.4603960396039604


: 